In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import math

In [2]:
# Jupyter configuration
pd.set_option('display.max_rows', 500)

In [3]:
dataset = pd.read_csv('owid-covid-data.csv')

In [4]:
dataset.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
count,1.207740e+05,120770.000000,119752.000000,1.097780e+05,109931.00000,119752.000000,120140.000000,120136.000000,119123.000000,109157.000000,...,87877.000000,86605.000000,56763.000000,102516.000000,121209.000000,112889.000000,4580.000000,4580.000000,4580.000000,4580.000000
mean,1.544711e+06,6460.637079,6482.477304,3.852423e+04,143.42483,131.157306,19239.874988,85.694527,85.756789,396.336812,...,10.591196,32.746339,50.875129,3.030193,73.258386,0.726279,30724.468472,8.455262,15.658900,755.471243
std,1.028266e+07,39146.952783,38748.402434,2.291702e+05,782.39257,733.454030,32106.971979,197.875181,167.394861,657.218700,...,10.503276,13.510481,31.823036,2.456243,7.533127,0.150053,86337.136808,15.822375,31.093173,1139.369805
min,1.000000e+00,-74347.000000,-6223.000000,1.000000e+00,-1918.00000,-232.143000,0.001000,-3125.829000,-272.971000,0.001000,...,0.100000,7.700000,1.188000,0.100000,53.280000,0.394000,-31959.400000,-27.350000,-95.920000,-1749.128494
25%,2.196000e+03,3.000000,9.714000,7.400000e+01,0.00000,0.000000,392.156000,0.292000,1.602500,11.422000,...,1.900000,21.600000,20.859000,1.300000,67.920000,0.602000,-186.075000,-1.090000,-0.990000,-46.895400
50%,2.375400e+04,94.000000,118.500000,6.530000e+02,2.00000,1.714000,3034.044500,10.972500,15.401000,76.080000,...,6.300000,31.400000,49.839000,2.400000,74.620000,0.744000,1992.150000,5.030000,6.275000,345.233709
75%,2.335360e+05,947.000000,997.464500,5.698500e+03,19.00000,16.000000,24206.247250,83.507750,93.350500,504.273000,...,19.300000,41.300000,83.241000,4.000000,78.740000,0.845000,19256.700000,13.440000,21.742500,1304.780877
max,2.471402e+08,905975.000000,826467.571000,5.006081e+06,18007.00000,14703.286000,230382.564000,8620.690000,3385.473000,6003.582000,...,44.000000,78.100000,100.000000,13.800000,86.750000,0.957000,754457.300000,106.830000,373.480000,6142.922470


In [5]:
# All available variables
dataset.columns.tolist()

['iso_code',
 'continent',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'total_tests',
 'total_tests_per_thousand',
 'new_tests_per_thousand',
 'new_tests_smoothed',
 'new_tests_smoothed_per_thousand',
 'positive_rate',
 'tests_per_case',
 'tests_units',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations',
 'new_vaccinations_smoothed',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'peo

In [6]:
# The size of the dataset
dataset.size

8308105

In [7]:
# Variable types
dataset.dtypes


iso_code                                    object
continent                                   object
location                                    object
date                                        object
total_cases                                float64
new_cases                                  float64
new_cases_smoothed                         float64
total_deaths                               float64
new_deaths                                 float64
new_deaths_smoothed                        float64
total_cases_per_million                    float64
new_cases_per_million                      float64
new_cases_smoothed_per_million             float64
total_deaths_per_million                   float64
new_deaths_per_million                     float64
new_deaths_smoothed_per_million            float64
reproduction_rate                          float64
icu_patients                               float64
icu_patients_per_million                   float64
hosp_patients                  

In [8]:
# fix data types WIP
dataset['date'] = pd.to_datetime(dataset['date'])